In [ ]:
import sglang as sgl
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

model_args = {
            "model_path": "Qwen/Qwen2-7B",
            "mem_fraction_static": 0.8,
            "trust_remote_code": False,
            "dtype": 'float16',
            "device": "cuda",
            "random_seed": 1234,
            "load_format": 'auto',
            "context_length": None,
            "dp_size": 1,
            "tp_size": 1,
            "log_level": "info",
        }

llm = sgl.Engine(**model_args)

[2025-02-13 18:54:33] server_args=ServerArgs(model_path='Qwen/Qwen2-7B', tokenizer_path='Qwen/Qwen2-7B', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda:7', served_model_name='Qwen/Qwen2-7B', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, host='127.0.0.1', port=30000, mem_fraction_static=0.8, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, stream_output=False, random_seed=1234, constrained_json_whitespace_pattern=None, watchdog_timeout=300, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metrics=False, decode_log_interval=40, api_key=None, file_storage_pth=

INFO 02-13 18:54:37 __init__.py:190] Automatically detected platform cuda.
INFO 02-13 18:54:37 __init__.py:190] Automatically detected platform cuda.


[2025-02-13 18:54:39 TP0] Casting torch.bfloat16 to torch.float16.
[2025-02-13 18:54:40 TP0] Casting torch.bfloat16 to torch.float16.
[2025-02-13 18:54:40 TP0] Init torch distributed begin.
[2025-02-13 18:54:40 TP0] Scheduler hit an exception: Traceback (most recent call last):
  File "/data/qiujiang/.local/lib/python3.10/site-packages/sglang/srt/managers/scheduler.py", line 1787, in run_scheduler_process
    scheduler = Scheduler(server_args, port_args, gpu_id, tp_rank, dp_rank)
  File "/data/qiujiang/.local/lib/python3.10/site-packages/sglang/srt/managers/scheduler.py", line 240, in __init__
    self.tp_worker = TpWorkerClass(
  File "/data/qiujiang/.local/lib/python3.10/site-packages/sglang/srt/managers/tp_worker_overlap_thread.py", line 63, in __init__
    self.worker = TpModelWorker(server_args, gpu_id, tp_rank, dp_rank, nccl_port)
  File "/data/qiujiang/.local/lib/python3.10/site-packages/sglang/srt/managers/tp_worker.py", line 68, in __init__
    self.model_runner = ModelRunner(

: 

In [ ]:
from sglang.srt.hf_transformers_utils import get_tokenizer

sgl_tokenizer = get_tokenizer(
               "Qwen/Qwen2-7B",
               tokenizer_mode="auto",
               trust_remote_code=False,
               tokenizer_revision="main",
            )
# input_text = "Question: Fred was preparing for a party to be held in four days.  So, he made 24 gallons of root beer on the first day and put them in the refrigerator cooler.  But later that evening, his children discovered the delicious nectar and robbed the cooler, drinking 4 of those gallons of root beer.  On the second day, his wife Barbie also discovered the root beer and accidentally spilled 7 gallons. On the third day, Fred's friend Ronnie visited Fred's house and helped himself to the root beer, further reducing the amount remaining by 5 gallons.  On the fourth day, 3 people showed up for the party.  If Fred and the others shared the remaining root beer equally, how much was available for each to drink during the party?\nAnswer:"
# input_text = "Question: A farmer is buying feed for his horses. He buys a variety of hay, oats, carrots and sugar cubes. Since sugar cubes are a rare treat, he only buys two 1-pound boxes of them for the whole stable. He only wants enough carrots to feed the horses while the vegetables are fresh, so he buys four 12-pound bags. Hay is the main diet of his horses, so he buys forty-two 75-pound bales. Oats are a staple to supplement the hay, so he buys twenty 65-pound sacks. If his farm truck can carry 2250 pounds at a time, how many trips does the farmer need to transport all the feed?\nAnswer:"
input_text = "Question: The great dragon, Perg, sat high atop mount Farbo, breathing fire upon anything within a distance of 1000 feet.  Polly could throw the gold javelin, the only known weapon that could sleigh the dragon, for a distance of 400 feet, well within the reach of the dragon's flames.  But when Polly held the sapphire gemstone, she could throw the javelin three times farther than when not holding the gemstone. If holding the gemstone, how far outside of the reach of the dragon's flames could Polly stand and still hit the dragon with the gold javelin?\nAnswer:"


input_tokens = sgl_tokenizer.encode(input_text)
print(f"Input Text: {input_text}")
print(f"Tokenized Input: {input_tokens}")

sgl_sampling_params = {
                  'max_new_tokens': 256, 
                  'stop_token_ids': [], 
                  'temperature': 1.0, 
                  'stop': ['Question=', 'Question', '=', '<|endoftext|>'], 
                  'top_p': 1.0, 
                  'top_k': -1, 
                  'min_p': 0.0, 
                  'frequency_penalty': 0.0, 
                  'presence_penalty': 0.0, 
                  'repetition_penalty': 1.0, 
                  'min_new_tokens': 0, 
                  'skip_special_tokens': True,
                  'n': 1
               }

outputs = llm.generate(input_ids=input_tokens, sampling_params=sgl_sampling_params, return_logprob=True)

meta_info = outputs["meta_info"]
output_token_logprobs = meta_info["output_token_logprobs"]
logprobs = [output[0] for output in output_token_logprobs]
print(outputs['text'])
print(logprobs)
print(outputs)

Input Text: Question: The great dragon, Perg, sat high atop mount Farbo, breathing fire upon anything within a distance of 1000 feet.  Polly could throw the gold javelin, the only known weapon that could sleigh the dragon, for a distance of 400 feet, well within the reach of the dragon's flames.  But when Polly held the sapphire gemstone, she could throw the javelin three times farther than when not holding the gemstone. If holding the gemstone, how far outside of the reach of the dragon's flames could Polly stand and still hit the dragon with the gold javelin?
Answer:
Tokenized Input: [14582, 25, 576, 2244, 25105, 11, 393, 2375, 11, 7578, 1550, 45988, 6470, 13452, 749, 11, 25938, 3940, 5193, 4113, 2878, 264, 6010, 315, 220, 16, 15, 15, 15, 7541, 13, 220, 93417, 1410, 2510, 279, 6623, 1258, 32730, 11, 279, 1172, 3881, 10288, 429, 1410, 19747, 1090, 279, 25105, 11, 369, 264, 6010, 315, 220, 19, 15, 15, 7541, 11, 1632, 2878, 279, 5545, 315, 279, 25105, 594, 38785, 13, 220, 1988, 979, 934

[2025-02-13 18:52:48 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 131, cache hit rate: 85.53%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-13 18:52:48 TP0] Decode batch. #running-req: 1, #token: 168, token usage: 0.00, gen throughput (token/s): 1.56, #queue-req: 0
[2025-02-13 18:52:49 TP0] Decode batch. #running-req: 1, #token: 208, token usage: 0.00, gen throughput (token/s): 90.40, #queue-req: 0


 The maximum strength of the dragon's flame is 1000 feet.
Polly can throw the javelin 400 feet.
With the gemstone, Polly can throw the javelin 1200 feet.
To know how far outside of the reach of the dragon's flames, subtract the radius of dragon's flame from the distance Polly can throw the javelin with the gemstone.
1000 feet - 1200 feet 
[-2.8583950996398926, -3.8593215942382812, -7.556135654449463, -0.5337253212928772, -0.3920714557170868, -0.4501698911190033, -0.915484607219696, -2.821345329284668, -0.59368497133255, -0.08475291728973389, -0.024543894454836845, -0.018635280430316925, -0.0005643171607516706, -0.0018791320035234094, -0.06897734850645065, -1.1276276111602783, -1.1253678798675537, -0.0005931228515692055, -0.3833225667476654, -0.05166562274098396, -0.10860683768987656, -0.219186469912529, -0.0021912604570388794, -0.49477970600128174, -0.05115484446287155, -0.0008736491436138749, -9.167614916805178e-05, -0.0061201793141663074, -1.7261691093444824, -1.6716943979263306, -0.

In [8]:

llm.shutdown()